In [7]:
from pygraphblas import *
        
udt = UDT('BF', ['double w', 'uint64_t h', 'uint64_t p'])

@udt.binop('min')
def bf_min(z, x, y):
    z.w = x.w if x.w < y.w else y.w

@udt.binop('plus')
def bf_plus(z, x, y):
    z.w = x.w + y.w

min_op = BinaryOp('min', 'BF', bf_min, udt)
plus_op = BinaryOp('plus', 'BF', bf_plus, udt)

udt.add_op = min_op
udt.mult_op = plus_op

A = Matrix.from_type(udt, 10, 10)
B = Matrix.from_type(udt, 10, 10)

A[1,1] = (0.9, 2, 9)
B[1,1] = (0.4, 2, 2)

with min_op:
    print((A + B).to_lists())

[[1], [1], [(0.4, 140735688701360, 0)]]


In [2]:
import ipdb; ipdb.pm()

AttributeError: module 'sys' has no attribute 'last_traceback'